In [1]:
import json
import string
import random
import nltk
import numpy as num
from nltk.stem import WordNetLemmatizer # It has the ability to lemmatize.
import tensorflow as tensorF # A multidimensional array of elements is represented by this symbol.
from tensorflow.keras import Sequential # Sequential groups a linear stack of layers into a tf.keras.Model
from tensorflow.keras.layers import Dense, Dropout

nltk.download("punkt")# required package for tokenization
nltk.download("wordnet")# word database

[nltk_data] Downloading package punkt to C:\Users\bhagya
[nltk_data]     sri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\bhagya
[nltk_data]     sri\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
data_file = open('C:\\Users\\bhagya sri\\Downloads\\Data.json').read()
data = json.loads(data_file)

data

{'ourIntents': [{'tag': 'age',
   'patterns': ['how old are you?'],
   'responses': ['I am 2 years old and my birthday was yesterday']},
  {'tag': 'greeting',
   'patterns': ['Hi', 'Hello', 'Hey'],
   'responses': ['Hi there', 'Hello', 'Hi :)']},
  {'tag': 'goodbye',
   'patterns': ['bye', 'later'],
   'responses': ['Bye', 'take care']},
  {'tag': 'name',
   'patterns': ["what's your name?", 'who are you?'],
   'responses': ['I have no name yet',
    'You can give me one, and I will appreciate it']},
  {'tag': 'exit',
   'patterns': ['quit', 'exit'],
   'responses': ['Okay, Good bye!!', 'Bye, Have a nice day ']},
  {'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day',
    'Namaste',
    'yo'],
   'responses': ['Hello',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Get lost',
  

In [3]:
lm = WordNetLemmatizer() #reducing words to their base or dictionary form

ourClasses = []
newWords = []
documentX = []
documentY = []
# Each intent is tokenized into words and the patterns and their associated tags are added to their respective lists.
for intent in data["ourIntents"]:
    for pattern in intent["patterns"]:
        ournewTkns = nltk.word_tokenize(pattern)
        newWords.extend(ournewTkns)
        documentX.append(pattern)
        documentY.append(intent['tag'])
    if intent["tag"] not in ourClasses:
        ourClasses.append(intent["tag"])

newWords = [lm.lemmatize(word.lower()) for word in newWords if word not in string.punctuation]
newWords = sorted(set(newWords))
ourClasses = sorted(set(ourClasses))

In [4]:
trainingData  = []
outEmpty = [0] * len(ourClasses)

for idx, doc in enumerate(documentX):
    bag0words = []
    text = lm.lemmatize(doc.lower())
    for word in newWords :
        bag0words.append(1) if word in text else bag0words.append(0)

    outputRow = list(outEmpty)
    outputRow[ourClasses.index(documentY[idx])] = 1 
    trainingData.append([bag0words, outputRow])

random.shuffle(trainingData)
trainingData = num.array(trainingData, dtype=object)

x = num.array(list(trainingData[:,0]))
y = num.array(list(trainingData[:,1]))

In [5]:
iShape = (len(x[0]),)
oShape = len(y[0])

Model = Sequential()

Model.add(Dense(128, activation="relu" , input_shape=iShape))

Model.add(Dropout(0.5)) #Dropout is a regularization technique that randomly drops out (sets to zero) some of the inputs to a layer during training to prevent overfitting.

Model.add(Dense(64, activation="relu"))

Model.add(Dropout(0.3))

Model.add(Dense(oShape, activation='softmax'))

md = tensorF.keras.optimizers.Adam(learning_rate= 0.01)

Model.compile(optimizer=md, loss='categorical_crossentropy', metrics=['accuracy'])

print(Model.summary())

Model.fit(x,y, epochs=200, verbose=1)

C:\Users\bhagya sri\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 128)                 │          16,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 32)                  │           2,080 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 26,976 (105.38 KB)

 Trainable params: 26,976 (105.38 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.0379 - loss: 3.4633   
Epoch 2/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0996 - loss: 3.2767 
Epoch 3/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2283 - loss: 3.0379 
Epoch 4/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1644 - loss: 2.7671 
Epoch 5/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3271 - loss: 2.4669 
Epoch 6/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4794 - loss: 2.0952 
Epoch 7/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5033 - loss: 1.9135 
Epoch 8/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6081 - loss: 1.4215 
Epoch 9/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6413 - loss: 1.2925 
Epoch 10/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6369 - loss: 1.2061 
Epoch 11/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7702 - loss: 0.8963 
Epoch 12/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.789

In [6]:
def ourText(text):
  newtkns = nltk.word_tokenize(text)
  newtkns = [lm.lemmatize(word) for word in newtkns]
  return newtkns

def wordBag(text, vocab):
  newtkns = ourText(text)
  bagOwords = [0] * len(vocab)
  for w in newtkns:
    for idx, word in enumerate(vocab):
      if word == w:
        bagOwords[idx] = 1
  return num.array(bagOwords)

def Pclass(text, vocab, labels):
  bagOwords = wordBag(text, vocab)
  ourResult = Model.predict(num.array([bagOwords]))[0]
  newThresh = 0.2
  yp = [[idx, res] for idx, res in enumerate(ourResult) if res > newThresh]

  yp.sort(key=lambda x: x[1], reverse=True)
  newList = []
  for r in yp:
    newList.append(labels[r[0]])
  return newList

def getRes(firstlist, fJson):
  tag = firstlist[0]
  listOfIntents = fJson["ourIntents"]
  for i in listOfIntents:
    if i["tag"] == tag:
      ourResult = random.choice(i["responses"])
      break
  return ourResult

In [ ]:
while True:
    newMessage = input("")
    intents = Pclass(newMessage, newWords, ourClasses)
    ourResult = getRes(intents, data)
    print(ourResult)

In [ ]:
while True:
    newMessage = input("")
    intents = Pclass(newMessage, newWords, ourClasses)
    ourResult = getRes(intents, data)
    print(ourResult)

what support is offered
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
I am a general purpose chatbot. My capabilities are : 
 1. I can chat with you. Try asking me for jokes or riddles! 
 2. Ask me the date and time 
 3. I can google search for you. Use format google: your query 
 4. I can get the present weather for any city. Use format weather: city name 
 5. I can get you the top 10 trending news in India. Use keywords 'Latest News' 
 6. I can get you the top 10 trending songs globally. Type 'songs' 
 7. I can set a timer for you. Enter 'set a timer: minutes to timer' 
 8. I can get the present Covid stats for any country. Use 'covid 19: world' or 'covid 19: country name' 
 For suggestions to help me improve, send an email to ted.thedlbot.suggestions@gmail.com . Thank you!! 
Make me laugh
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
I own the world's worst thesaurus. Not only is it awful, it's awful.
Hi there
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Hello
Hey
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Hel